### **Classificació amb Transfer Learning utilitzant VCC16**

En aquest codi, implementem un model de classificació de fruites utilitzant transfer learning amb l’arquitectura preentrenada VGG16.

#### **Característiques clau**
- **VGG16 preentrenat:** S’utilitza com a base amb els pesos d’ImageNet, eliminant la capa superior (`include_top=False`) per adaptar-lo al nostre problema.
- **Capacitat d'aprenentatge congelada:** Les capes de VGG16 es mantenen no entrenables (`layer.trainable = False`) per aprofitar les característiques preentrenades.
- **Cap personalitzada:** S’afegeixen capes al final:
  - `GlobalAveragePooling2D`: Per reduir la dimensionalitat.
  - `Dense` amb 1024 unitats i activació `relu` per afegir complexitat.
  - `Dense` amb 6 unitats i activació `softmax` per a la classificació de les 6 classes.

#### **Generació de dades**
- Utilitzem `ImageDataGenerator` per generar dades augmentades:
  - Escalem els píxels a un rang [0, 1].
  - Apliquem transformacions com rotacions, desplaçaments, zooms i reflexions horitzontals per evitar sobreajustament.
- Les dades es carreguen en carpetes separades per a entrenament i validació.

#### **Entrenament**
- El model es compila amb `categorical_crossentropy` com a funció de pèrdua i `Adam` com a optimitzador.
- Entrenem durant 10 èpoques utilitzant les dades augmentades.

#### **Objectiu**
Aprofitar el transfer learning amb VGG16 per millorar la classificació multiclasse de fruites i comparar els resultats amb els models SVM anteriors.


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# Ruta a las carpetas de entrenamiento y validación
train_dir = '.gitignore/dataset/train'
test_dir = '.gitignore/dataset/test'

# Usar VGG16 preentrenado sin la capa superior
vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Congelar las capas del modelo base
for layer in vgg_base.layers:
    layer.trainable = False

# Añadir nuestras capas personalizadas
x = vgg_base.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(6, activation='softmax')(x)  # 6 clases en total: freshapple, freshbanana, freshorange, rottenapple, rottenbanana, rottenorange

# Crear el modelo final
model = Model(inputs=vgg_base.input, outputs=x)

# Compilar el modelo
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Crear generadores de datos para entrenar y validar
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Escalar los píxeles a un rango [0, 1]
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

# Cargar las imágenes de entrenamiento
train_generator = train_datagen.flow_from_directory(
    train_dir,  # Ruta de entrenamiento
    target_size=(224, 224),  # Tamaño para VGG16
    batch_size=32,
    class_mode='categorical'  # Como es clasificación multiclase
)

# Cargar las imágenes de validación
validation_generator = val_datagen.flow_from_directory(
    test_dir,  # Ruta de prueba
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Entrenar el modelo
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

# Guardar el modelo entrenado
model.save('fruit_classification_model.h5')

### **Avaluació del model amb dades de prova**

En aquest codi, avaluem el model entrenat utilitzant el conjunt de dades de prova i generem mètriques per analitzar-ne el rendiment.

#### **Característiques clau**
- **Generador de dades de validació:** Es carrega el conjunt de prova amb `ImageDataGenerator` i es manté l'ordre de les etiquetes (`shuffle=False`) per garantir que les prediccions coincideixin amb les etiquetes reals.
- **Prediccions:** Utilitzem el model carregat (`fruit_classification_model.h5`) per generar prediccions sobre el conjunt de prova.
- **Mètriques d’avaluació:**
  - **Informe de classificació (`classification_report`):** Mostra mètriques com precisió, recall i F1-score per cada classe.
  - **Matriiu de confusió:** Identifica on es produeixen errors en les prediccions, visualitzant les etiquetes reals vs. prediccions.

#### **Visualització**
- La **matriz de confusió** es representa gràficament amb un mapa de calor (`heatmap`) que facilita identificar les classes amb millor i pitjor rendiment.

#### **Objectiu**
Avaluar la precisió i consistència del model en dades reals de prova i identificar possibles millores per optimitzar el rendiment.


In [ ]:
import tensorflow as tf
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Directorio de prueba
test_dir = '.gitignore/dataset/test'

# Crear generador de datos para validación
val_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = val_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False  # Importante para mantener el orden de las etiquetas
)

# Cargar el modelo entrenado
model = tf.keras.models.load_model('fruit_classification_model.h5')

# Generar predicciones con el conjunto de validación
validation_generator.reset()  # Reiniciar el generador
predictions = model.predict(validation_generator, steps=validation_generator.samples // validation_generator.batch_size + 1)
predicted_classes = np.argmax(predictions, axis=1)

# Obtener las etiquetas reales
true_classes = validation_generator.classes
class_labels = list(validation_generator.class_indices.keys())

# Generar el informe de clasificación
report = classification_report(true_classes, predicted_classes, target_names=class_labels)
print("Informe de Clasificación:")
print(report)

# Matriz de confusión
conf_matrix = confusion_matrix(true_classes, predicted_classes)

# Visualizar la matriz de confusión
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=class_labels, yticklabels=class_labels)
plt.title('Matriz de Confusión')
plt.ylabel('Etiqueta Real')
plt.xlabel('Etiqueta Predicha')
plt.show()
